In [45]:
import pandas as pd
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

pd.set_option('display.max_rows', None)

db_path = 'deals_db.db'

In [46]:
def clean_text(text):
    cleaned_text = re.sub(r'\b[a-zA-Z]\b|\b\d+\b', '', text)
    return re.sub(r'\s+', ' ', cleaned_text).strip().lower()

In [48]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""SELECT c.title || " " ||  c.title_general as document
    FROM deals c
""")
rows = cursor.fetchall()

conn.close()

In [52]:
corpus = []
for row in rows:
    if row[0]:
        line = clean_text(row[0])
        corpus.append(line)

In [53]:
corpus[:2]

['sunset tour by kayak on sebago lake maine sunset tour by kayak on sebago lake maine',
 'quarter rotisserie chicken meal savor the flavor: quarter rotisserie chicken meal at texas rotisserie and grill- broadway(up to % off)']

In [55]:
tokens = []
for line in corpus:
    data = word_tokenize(line)
    for word in data:
    #filtered_data = [[word for word in sentence if len(word) > 2] for sentence in data]
        if len(word) > 2:
            tokens.append(word)

In [ ]:
#documents = [' '.join(sentence) for sentence in tokens]
documents = [' '.join(tokens)]

# Inicializace TfidfVectorizer
#tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

# Fit and transform data
#tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Get feature names
#trigrams = tfidf_vectorizer.get_feature_names_out()

# Compute the mean TF-IDF scores
#mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()

#filtered_mean_tfidf = [(trigram, tfidf) for trigram, tfidf in zip(trigrams, mean_tfidf) if tfidf > 0.000001]


# Save to CSV
#output_file = 'models/mean_tfidf_unigram.csv'
#with open(output_file, 'w', encoding='utf-8') as f:
#    f.write('unigram,tfidf\n')
#    for trigram, tfidf in filtered_mean_tfidf:
#       f.write(f'{trigram},{tfidf}\n')
documents[:10]

In [8]:
df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
df.count()

unigram    51009
tfidf      51009
dtype: int64

In [89]:
filtered_mean_tfidf[:10]

[('002oz57ml', 1.5954199494191315e-05),
 ('003oz11g', 1.5954199494191315e-05),
 ('004oz11g', 4.786259848257395e-05),
 ('007oz22g', 1.5954199494191315e-05),
 ('008oz24g', 1.5954199494191315e-05),
 ('010oz3g', 1.5954199494191315e-05),
 ('012oz35g', 1.5954199494191315e-05),
 ('015ct', 2.3931299241286974e-05),
 ('015mm', 2.3931299241286974e-05),
 ('016oz52ml', 1.5954199494191315e-05)]

In [ ]:
df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
df.count()
dictionary = df["unigram"].tolist()
dictionary

In [107]:
from fuzzywuzzy import process

dictionary = df["unigram"].tolist()
for w in ["valvoln", "mesage"]:
    corrected_word = process.extractBests(w, dictionary)
    print(corrected_word[0]) # 

('vol', 90)
('message', 92)


In [101]:
from transformers import pipeline

/Users/zphilipp/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [30]:
df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
df.count()
dictionary = df["unigram"].tolist()


In [42]:
def correct_typo(input_word):
    # Tokenizace vstupního slova
    input_words = [input_word]


    df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
    df.count()
    dictionary = df["unigram"].tolist()
    # Vytvoření TF-IDF vektorizeru
    vectorizer = TfidfVectorizer().fit(input_words + dictionary)
    
    # Vytvoření vektorů pro vstupní slovo a slova ve slovníku
    input_vector = vectorizer.transform(input_words)
    

    dictionary_vectors = vectorizer.transform(dictionary)

    # Výpočet cosine similarity
    similarities = cosine_similarity(input_vector, dictionary_vectors)

    # Najděte index slova s nejvyšší podobností
    most_similar_index = np.argmax(similarities)

    # Vrátit slovo ze slovníku s nejvyšší podobností
    return dictionary_words[most_similar_index]

# Příklad použití
input_word = "massage"  # Překlep

corrected_word = correct_typo(input_word)
print(f"Opravené slovo pro '{input_word}' je '{corrected_word}'.")

IndexError: list index out of range